In [ ]:
%matplotlib

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [2]:
import numpy as np

from python.fluorophores import FlStatic
from python.psfs import PsfVectorial
from python.estimators import est_donut2d
from python.simulators import Simulator
from python.mProfile import mProfile

In [3]:
fl = FlStatic()  # define a static fluorophore
fl.pos = [10, 0, 0]
fl.brightness = 1000 # kHz if excited at the center of a Gaussian beam

psf_vec = PsfVectorial() 

sim = Simulator(fl)

numberOfLocalizations=1000

In [4]:
# define scan pattern
L = 75  # size of scan pattern
orbitpoints = 6  # number of probing points in orbit
probecenter = True  # should we also probe the center?
laserpower = 5  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 2 # how often to repeat the pattern scan
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [5]:
# we need an estimator. Define as component
fwhm=360  # size of the donut, needed for proper estimation. 
sim.defineComponent("estdonut", "estimator", est_donut2d, parameters=[sim.patterns["donut"].pos, L, fwhm], dim=(0,1))

In [ ]:
seq = ["donut", "estdonut"]
mProfile.profileOn(['utilities.py', 'psf.py', 'psf_vectorial.py', 'simulator.py'])

out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
mProfile.profileOff()
mProfile.report()

# sigmaCRB=sim.calculateCRB("donut", dim=(0,1))/np.sqrt(np.mean(out.loc.phot))

# # disp('vectorial PSF:')
# sim.summarize_results(out)

# psf0=psf_vec.imagestack("vortex")